In [ ]:
import redis
import random
import string
import json
from redis.sentinel import Sentinel

In [ ]:
# sentinel configuration
"""
the conf 

list of tuples containing [(host, port)]

"""
REDIS_HOST = "0.0.0.0"
REDIS_HOST_PORT_1 = "26379"
REDIS_HOST_PORT_2 = "26380"
REDIS_HOST_PORT_3 = "26381"
REDIS_PASSWORD = "password"
# REDIS_PASSWORD = "yourpassword"
# REDIS_USERNAME = "userone"
REDIS_USERNAME = "default"

kwargs = {}
password = kwargs["password"] = REDIS_PASSWORD
username = kwargs["username"] = REDIS_USERNAME


sentinel = Sentinel(
    [
        (REDIS_HOST, REDIS_HOST_PORT_1),
        (REDIS_HOST, REDIS_HOST_PORT_2),
        (REDIS_HOST, REDIS_HOST_PORT_3),
    ],
    **kwargs,
    socket_timeout=0.1,
)

In [ ]:
sentinel.discover_slaves(service_name="mymaster")

In [ ]:
host, port = sentinel.discover_master(service_name="mymaster")

In [ ]:
master = sentinel.master_for(
    service_name="mymaster",
)

In [ ]:
"""Test data"""
data = {
    "sofa": [("one", 1, 1), ("one", 15, 12), ("one", 5, 8), ("one", 1, 2)],
    "couch": [("one", 1, 1), ("one", 15, 12), ("one", 5, 8), ("one", 1, 2)],
    "fridge": [("one", 1, 2),("two", 1, 2), ("three", 1, 2)],
    "car": [("one", 1, 1), ("two", 15, 12), ("three", 5, 8), ("four", 1, 2)],
    "mops": [("one", 1, 4)],
    "vacum_cleaners": [("one", 1, 5)],
}

In [ ]:
"""
Process the test data into proper list of dictionaries and 
assigning the keys to the appropriate values.
"""
final_results_list = []

for key, value in data.items():
    dict = {}
    for tuple in value:
        dict["name"] = key
        dict["sku"] = tuple[0]
        dict["category_id"] = tuple[1]
        dict["score"] = tuple[2]
        final_results_list.append(dict)

In [ ]:
len(final_results_list)

In [ ]:
"""for each dictionary in the list a new key is generated and added to the redis sentinel (master node)"""
for dictionary  in final_results_list:
    # create a key for each dictionary
    # using the name as the key
    key = dictionary['name']

    # iterate over the items in the dictionary
    # and add them to a hash at the key
    for k, v in dictionary.items():
        master.hset(key, k, v)

In [ ]:
"""used to check keys."""
for key in master.scan_iter("*"):
    print(key)


